In [ ]:
!pip install paddlepaddle
!curl https://gist.githubusercontent.com/lime-j/f9e253332c87881df229eb09ec528e66/raw/b2bbfed2fc20b309e1fe8b9df2e04406ca46025f/esnet.py > esnet.py
!curl https://gist.githubusercontent.com/lime-j/a5d5383aa7031d0f33c0e2e9476ca9ec/raw/e32e6c0edafe95055d3ccb3e4e6e3bac6c4d0f25/pd_esnet.py > pd_esnet.py

In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleDetection

In [ ]:
%cd /content/PaddleDetection
!cp ./ppdet ../ -r
!pip install -r requirements.txt
%cd /content/

In [ ]:
from esnet import ESNet as ptesnet
from pd_esnet import ppESNet as ppesnet
import numpy as np
import paddle
import pickle
import torch

In [5]:
ptnet = ptesnet(scale=1.0,feature_maps=[4, 11, 14],channel_ratio=[0.875, 0.5, 1.0, 0.625, 0.5, 0.75, 0.625, 0.625, 0.5, 0.625, 1.0, 0.625, 0.75])
ppnet = ppesnet(scale=1.0,feature_maps=[4, 11, 14],channel_ratio=[0.875, 0.5, 1.0, 0.625, 0.5, 0.75, 0.625, 0.625, 0.5, 0.625, 1.0, 0.625, 0.75])

In [6]:
pdparams_root = "/content/model_final.pdparams"
pd_state_dict = paddle.load(pdparams_root)
pd_backbone_state_dict = paddle.load(pdparams_root) 
from_block_lst = [3, 7, 3]
pfx = [0, 3, 10]
for k in pd_state_dict.keys():
    if ("backbone" not in k) : continue
    to_key = str.replace(k, "backbone.", "backbone0.")
    to_key = str.replace(to_key, "_mean", "running_mean")
    to_key = str.replace(to_key, "_variance", "running_var")

    for i in range(2, 5):
        for j in range(1, from_block_lst[i - 2] + 1):
            if 'backbone0.%d_%d' % (i, j) in to_key:
                to_key = str.replace(to_key, 'backbone0.%d_%d' % (i, j), 'backbone0._block_list.%d' %(pfx[i - 2] + j - 1))

    to_key = str.replace(to_key, "backbone0.", "")
    ret_state_dict[to_key] = pd_state_dict[k].numpy()

    # Convert origin checkpoint into a backbone-only one
    to_pp_backbone_key = str.replace(k, "backbone.", "")
    pp_state_dict[to_pp_backbone_key] = pd_backbone_state_dict[k]  
    
for k in ret_state_dict.keys():
    ret_state_dict[k] = torch.from_numpy(ret_state_dict[k])

In [7]:
ptnet.load_state_dict(ret_state_dict)
ppnet.load_dict(pd_backbone_state_dict)   
ppnet.eval()
ptnet = ptnet.eval() 

In [8]:
check_ret = 0
for tests in range(20):
    x_np = np.random.randn(1, 3, 416, 416).astype(np.float32)
    x_pp = paddle.to_tensor(x_np)
    pp_ret = ppnet(x_pp)
    pt_ret = ptnet(torch.tensor(x_np))
    pp_ret = [x.numpy() for x in pp_ret]
    pt_ret = [x.detach().numpy() for x in pt_ret]
    for i in range(3):
        check_ret += np.mean(np.abs(pt_ret[i] - pp_ret[i]))
check_ret /= 20 * 3
assert check_ret <= 1e-3, "difference between paddle and pytorch is too large, suggesting something wrong happened"
print(check_ret)

1.9692649431135578e-07


In [10]:
torch.save(ptnet.state_dict(), "esnet-m-416-300epoch.pth")